In [9]:
import nest_asyncio
import asyncio

nest_asyncio.apply()

async def main():
    async with AsyncWebCrawler() as crawler:
        result = await crawler.arun(
            url="https://studerendeonline.dk/legate",
        )
        print(result.markdown)

await main()

[INIT].... → Crawl4AI 0.4.248
[FETCH]... ↓ https://studerendeonline.dk/legate... | Status: True | Time: 0.01s
[COMPLETE] ● https://studerendeonline.dk/legate... | Status: True | Total: 0.02s
×
####  Log ind
![](https://studerendeonline.dk/images/opengraph/jobbank-15.jpg)
×
#### Opret karriereprofil
Fortæl os lidt om dig selv og hjælp os med at matche dig med relevante job og karrieretilbud.
  * 1Uddannelse 
  * 2Dig selv 
  * 3Præferencer 


Hvad er din _højeste_ uddannelse? Er du fortsat under uddannelse, angiv gerne din højeste forventede titel samt anslået dimissionsdato. 
Uddannelsestitel * Anden erhvervsuddannelseAnden kort videregående udd.Anden lang videregående udd.Anden mellemlang videregående udd.Kurser, efteruddannelse og andetUdvekslingsopholdAdministrationsbachelorAdministrationsøkonomAdvokatAgriculture and EnvironmentAkademiingeniørAkademimerkonomAkademiuddannelse i HRAkademiuddannelse i ledelseAkademiøkonomAnimationsinstruktørAntropologAP Automotive TechnologyAP Chemical

In [1]:
from crawl4ai import AsyncWebCrawler, CacheMode, CrawlerRunConfig, JsonCssExtractionStrategy
import json
import polars as pl
import asyncio


# Define the async function to extract scholarships
async def extract_scholarships():
    # Define the extraction schema
    schema = {
        "name": "Scholarships Extractor",
        "baseSelector": "div.card",
        "fields": [
            {
                "name": "scholarship_url",
                "selector": "div.card-block a.btn.btn-primary.btn-sm.btn-slim.pull-right",
                "type": "attribute",
                "attribute": "href",
            },
        ],
    }

    extraction_strategy = JsonCssExtractionStrategy(schema, verbose=True)

    # Configure the crawler
    config = CrawlerRunConfig(
        cache_mode=CacheMode.BYPASS,
        extraction_strategy=extraction_strategy,
    )

    all_scholarships = []

    # Iterate over pages 1 to 5
    for page in range(1, 6):
        url = f"https://studerendeonline.dk/legater/find/?cat=4&page={page}"
        async with AsyncWebCrawler() as crawler:
            result = await crawler.arun(
                url=url,
                config=config
            )
            # Parse the extracted content for each page
            scholarships = json.loads(result.extracted_content)
            all_scholarships.extend(scholarships)
            print(f"Extracted URLs from page {page}:")
            for scholarship in scholarships:
                print(scholarship["scholarship_url"])
            print(f"\nSuccessfully extracted {len(scholarships)} scholarship URLs from page {page}.")

    # Save the results in a dataframe
    df_scholarships = pl.DataFrame(all_scholarships)
    print(df_scholarships)
    return df_scholarships

# Run the extraction and save the DataFrame
async def main():
    global df_scholarships
    try:
        df_scholarships = await extract_scholarships()
    except Exception as e:
        print(f"An error occurred: {e}")

# Run the async function in Jupyter Notebook
if __name__ == "__main__":
    asyncio.run(main())


RuntimeError: asyncio.run() cannot be called from a running event loop

In [21]:
df_scholarships

scholarship_url
str
"""/legater/10490/generelle-forma…"
"""/legater/10491/studieophold-fo…"
"""/legater/5041/studieophold-i-u…"
"""/legater/25276/studieophold-i-…"
"""/legater/10215/studie-og-prakt…"
…
"""/legater/26559/danske-kvinder-…"
"""/legater/9431/unge-med-tilknyt…"
"""/legater/9926/studerende-ved-k…"


In [22]:
# Assume df_scholarships is your polars DataFrame
base_url = "https://studerendeonline.dk"

# Concatenate the base URL with the partial URLs
df_scholarships = df_scholarships.with_columns(
    (base_url + df_scholarships['scholarship_url']).alias('full_url')
)

In [23]:
df_scholarships['full_url'].head(5).to_list()

['https://studerendeonline.dk/legater/10490/generelle-formal-herunder-kulturelle-og-sociale-formal',
 'https://studerendeonline.dk/legater/10491/studieophold-for-studerende-pa-kunstneriske-uddannelser',
 'https://studerendeonline.dk/legater/5041/studieophold-i-udlandet-kandidatniveau',
 'https://studerendeonline.dk/legater/25276/studieophold-i-udlandet-sygeplejestuderende',
 'https://studerendeonline.dk/legater/10215/studie-og-praktikophold-i-udlandet-for-alle-studieretninger']

In [24]:
df_scholarships['full_url']

full_url
str
"""https://studerendeonline.dk/le…"
"""https://studerendeonline.dk/le…"
"""https://studerendeonline.dk/le…"
"""https://studerendeonline.dk/le…"
"""https://studerendeonline.dk/le…"
…
"""https://studerendeonline.dk/le…"
"""https://studerendeonline.dk/le…"
"""https://studerendeonline.dk/le…"


In [25]:
from crawl4ai import AsyncWebCrawler, CacheMode, CrawlerRunConfig
from crawl4ai.extraction_strategy import JsonCssExtractionStrategy
import json
import pandas as pd
import asyncio
import nest_asyncio

# Set up asyncio for Jupyter Notebook
nest_asyncio.apply()

async def extract_details(urls):
    schema = {
        "name": "LegatExtractor",
        "baseSelector": "body",
        "fields": [
            {
                "name": "Legat",
                "selector": "h1.page-title",
                "type": "text",
                "transform": "strip"
            },
            {
                # Teaser text specifically in div.teaser.read-more
                "name": "Beskrivelse",
                "selector": "div.teaser.read-more",
                "type": "text",
                "transform": "strip"
            },
            {
                # Specifically targets the .textbox.box-blue that has a grantForwarder button
                # This ensures we get the correct box with the 'Ansøg via Legatbogen' button
                "name": "Frist",
                "selector": "div.textbox.box-blue.text-small:has(button.grantForwarder) b.text-bold",
                "type": "text",
                "transform": "strip"
            },
            {
                # Everything else (“Vi støtter” details, h2, etc.) 
                # is inside .col-sm-7.col-md-8
                # If you only want the row portion, use '> .row' 
                # or a more precise child selector
                "name": "Detaljer",
                "selector": "div.col-sm-7.col-md-8 > .row",
                "type": "text",
                "transform": "strip"
            }
        ]
    }
    
    extraction_strategy = JsonCssExtractionStrategy(schema, verbose=True)

    config = CrawlerRunConfig(
        cache_mode=CacheMode.BYPASS,
        extraction_strategy=extraction_strategy,
    )

    all_results = []

    async with AsyncWebCrawler() as crawler:
        for url in urls:
            result = await crawler.arun(url=url, config=config)
            if result.extracted_content is None:
                print(f"No content extracted from {url}")
                continue
            try:
                data = json.loads(result.extracted_content)
                all_results.append(data)
            except json.JSONDecodeError:
                print(f"Failed to decode JSON from {url}")

    return all_results

async def process_urls():
    # Adjust the DataFrame, column name, and slice as needed
    sample_urls = df_scholarships['full_url'].head(5).to_list()
    
    details_list = await extract_details(sample_urls)
    details_df = pd.DataFrame(details_list)
    return details_df

result_df = asyncio.run(process_urls())
print(result_df)


[INIT].... → Crawl4AI 0.4.248
[FETCH]... ↓ https://studerendeonline.dk/legater/10490/generell... | Status: True | Time: 6.63s
[SCRAPE].. ◆ Processed https://studerendeonline.dk/legater/10490/generell... | Time: 615ms
[EXTRACT]. ■ Completed for https://studerendeonline.dk/legater/10490/generell... | Time: 2.6132866669940995s
[COMPLETE] ● https://studerendeonline.dk/legater/10490/generell... | Status: True | Total: 10.07s
[FETCH]... ↓ https://studerendeonline.dk/legater/10491/studieop... | Status: True | Time: 8.79s
[SCRAPE].. ◆ Processed https://studerendeonline.dk/legater/10491/studieop... | Time: 2323ms
[EXTRACT]. ■ Completed for https://studerendeonline.dk/legater/10491/studieop... | Time: 5.948104959010379s
[COMPLETE] ● https://studerendeonline.dk/legater/10491/studieop... | Status: True | Total: 17.24s
[FETCH]... ↓ https://studerendeonline.dk/legater/5041/studieoph... | Status: True | Time: 5.78s
[SCRAPE].. ◆ Processed https://studerendeonline.dk/legater/5041/studieoph... | Time: 7

In [26]:
from crawl4ai import AsyncWebCrawler, CacheMode, CrawlerRunConfig
from crawl4ai.extraction_strategy import JsonCssExtractionStrategy
import json
import asyncio
import nest_asyncio

# Set up asyncio for Jupyter Notebook
nest_asyncio.apply()

async def extract_details(urls):
    schema = {
        "name": "LegatExtractor",
        "baseSelector": "body",
        "fields": [
            {
                "name": "Legat",
                "selector": "h1.page-title",
                "type": "text",
                "transform": "strip"
            },
            {
                # Teaser text specifically in div.teaser.read-more
                "name": "Beskrivelse",
                "selector": "div.teaser.read-more",
                "type": "text",
                "transform": "strip"
            },
            {
                # Specifically targets the .textbox.box-blue that has a grantForwarder button
                # This ensures we get the correct box with the 'Ansøg via Legatbogen' button
                "name": "Frist",
                "selector": "div.textbox.box-blue.text-small:has(button.grantForwarder) b.text-bold",
                "type": "text",
                "transform": "strip"
            },
            {
                # Everything else (“Vi støtter” details, h2, etc.) 
                # is inside .col-sm-7.col-md-8
                # If you only want the row portion, use '> .row' 
                # or a more precise child selector
                "name": "Detaljer",
                "selector": "div.col-sm-7.col-md-8 > .row",
                "type": "text",
                "transform": "strip"
            }
        ]
    }
    
    extraction_strategy = JsonCssExtractionStrategy(schema, verbose=True)

    config = CrawlerRunConfig(
        cache_mode=CacheMode.BYPASS,
        extraction_strategy=extraction_strategy,
    )

    all_results = []

    async with AsyncWebCrawler() as crawler:
        for url in urls:
            result = await crawler.arun(url=url, config=config)
            if result.extracted_content is None:
                print(f"No content extracted from {url}")
                continue
            try:
                data = json.loads(result.extracted_content)
                all_results.append(data)
            except json.JSONDecodeError:
                print(f"Failed to decode JSON from {url}")

    return all_results

async def process_urls():
    # Adjust the DataFrame, column name, and slice as needed.
    # Here, assuming df_scholarships is defined elsewhere.
    sample_urls = df_scholarships['full_url'].head(5).to_list()
    
    details_list = await extract_details(sample_urls)
    return details_list

# Run the asynchronous processing and get the JSON results
results = asyncio.run(process_urls())

# Save the JSON data to a file
with open('results.json', 'w', encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

# Optionally, print the JSON results
print(json.dumps(results, ensure_ascii=False, indent=2))


[INIT].... → Crawl4AI 0.4.248
[FETCH]... ↓ https://studerendeonline.dk/legater/10490/generell... | Status: True | Time: 8.57s
[SCRAPE].. ◆ Processed https://studerendeonline.dk/legater/10490/generell... | Time: 1796ms
[EXTRACT]. ■ Completed for https://studerendeonline.dk/legater/10490/generell... | Time: 4.015203792005195s
[COMPLETE] ● https://studerendeonline.dk/legater/10490/generell... | Status: True | Total: 15.95s
[FETCH]... ↓ https://studerendeonline.dk/legater/10491/studieop... | Status: True | Time: 7.41s
[SCRAPE].. ◆ Processed https://studerendeonline.dk/legater/10491/studieop... | Time: 4292ms
[EXTRACT]. ■ Completed for https://studerendeonline.dk/legater/10491/studieop... | Time: 4.578331290991628s
[COMPLETE] ● https://studerendeonline.dk/legater/10491/studieop... | Status: True | Total: 18.11s
[FETCH]... ↓ https://studerendeonline.dk/legater/5041/studieoph... | Status: True | Time: 5.70s
[SCRAPE].. ◆ Processed https://studerendeonline.dk/legater/5041/studieoph... | Time: 2

In [ ]:
from crawl4ai import AsyncWebCrawler, CacheMode, CrawlerRunConfig
from crawl4ai.extraction_strategy import JsonCssExtractionStrategy
import json
import pandas as pd
import asyncio
import nest_asyncio

# Set up asyncio for Jupyter Notebook
nest_asyncio.apply()

async def extract_details(urls):
    schema = {
        "name": "LegatExtractor",
        "baseSelector": "body",
        "fields": [
            {
                "name": "Legat",
                "selector": "h1.page-title",
                "type": "text",
                "transform": "strip"
            },
            {
                # Teaser text specifically in div.teaser.read-more
                "name": "Beskrivelse",
                "selector": "div.teaser.read-more",
                "type": "text",
                "transform": "strip"
            },
            {
                # Specifically targets the .textbox.box-blue that has a grantForwarder button
                # This ensures we get the correct box with the 'Ansøg via Legatbogen' button
                "name": "Frist",
                "selector": "div.textbox.box-blue.text-small:has(button.grantForwarder) b.text-bold",
                "type": "text",
                "transform": "strip"
            },
            {
                # Everything else (“Vi støtter” details, h2, etc.) 
                # is inside .col-sm-7.col-md-8
                # If you only want the row portion, use '> .row' 
                # or a more precise child selector
                "name": "Detaljer",
                "selector": "div.col-sm-7.col-md-8 > .row",
                "type": "text",
                "transform": "strip"
            }
        ]
    }
    
    extraction_strategy = JsonCssExtractionStrategy(schema, verbose=True)

    config = CrawlerRunConfig(
        cache_mode=CacheMode.BYPASS,
        extraction_strategy=extraction_strategy,
    )

    all_results = []

    async with AsyncWebCrawler() as crawler:
        for url in urls:
            result = await crawler.arun(url=url, config=config)
            if result.extracted_content is None:
                print(f"No content extracted from {url}")
                continue
            try:
                data = json.loads(result.extracted_content)
                all_results.append(data)
            except json.JSONDecodeError:
                print(f"Failed to decode JSON from {url}")

    return all_results

async def process_urls():
    
    details_list = await extract_details(df_scholarships['full_url'].to_list())
    details_df = pd.DataFrame(details_list)
    return details_df

result_df = asyncio.run(process_urls())
print(result_df)


[INIT].... → Crawl4AI 0.4.248
[FETCH]... ↓ https://studerendeonline.dk/legater/10490/generell... | Status: True | Time: 9.47s
[SCRAPE].. ◆ Processed https://studerendeonline.dk/legater/10490/generell... | Time: 605ms
[EXTRACT]. ■ Completed for https://studerendeonline.dk/legater/10490/generell... | Time: 0.44127433299581753s
[COMPLETE] ● https://studerendeonline.dk/legater/10490/generell... | Status: True | Total: 10.64s
[FETCH]... ↓ https://studerendeonline.dk/legater/10491/studieop... | Status: True | Time: 6.16s
[SCRAPE].. ◆ Processed https://studerendeonline.dk/legater/10491/studieop... | Time: 545ms
[EXTRACT]. ■ Completed for https://studerendeonline.dk/legater/10491/studieop... | Time: 0.432679833000293s
[COMPLETE] ● https://studerendeonline.dk/legater/10491/studieop... | Status: True | Total: 7.20s
[FETCH]... ↓ https://studerendeonline.dk/legater/5041/studieoph... | Status: True | Time: 6.72s
[SCRAPE].. ◆ Processed https://studerendeonline.dk/legater/5041/studieoph... | Time: 92

ModuleNotFoundError: No module named 'openpyxl'

In [16]:
import json
import pandas as pd

# Convert the first row to JSON
first_row_json = result_df.iloc[4].to_json()

# Print the JSON
print(json.dumps(json.loads(first_row_json), indent=4, ensure_ascii=False))

# Convert the JSON string back to a dictionary
first_row_dict = json.loads(first_row_json)

# Create a DataFrame from the dictionary
first_row_df = pd.DataFrame([first_row_dict])

# Display the DataFrame
print(first_row_df)

{
    "0": {
        "Legat": "William Demant Fonden",
        "Beskrivelse": "Fondens formål er, af fondens årlige nettoindtægt, at yde støtte til projekter, der har til formål at helbrede eller afhjælpe tunghørhed eller følgerne deraf.At yde støtte til fremme af erhvervsmæssige, kulturelle, videnskabelige, sociale, uddannelsesmæssige og kunstneriske formål og navnlig med sigte på at bidrage til dansk ungdoms uddannelse.At yde støtte efter bestyrelsens skøn til personer, der hører eller har hørt til den personkreds, der dannes af medarbejdere, ledelse og rådgivere for fonden eller de virksomheder, der måtte være tilknyttet fonden, samt disses nærmeste familie. Så længe ledelsen og dens rådgivere fungerer som sådan, vil der dog ikke kunne ydes personlig støtte til dem.",
        "Frist": "Løbende",
        "Detaljer": "Vi støtterStuderende med et karaktergennemsnit på mindst 7 efter 12-skalaenAlle studieretningerStuderende, der har færdiggjort deres bachelorStudieophold i udlandetKun s

In [49]:
first_row_df

,0
0,"{'Legat': 'William Demant Fonden', 'Beskrivels..."


In [18]:
details_list

NameError: name 'details_list' is not defined

# Qdrant vector database setup
Link ("https://qdrant.tech/documentation/beginner-tutorials/search-beginners/")

In [5]:
from dotenv import load_dotenv, find_dotenv
from qdrant_client import QdrantClient
import os

load_dotenv(find_dotenv())

Qdrant_api_key = os.getenv("Qdrant_api_key")
client = QdrantClient(
    url="https://2c49a62c-f2c9-4788-808d-63746748ccb5.eu-central-1-0.aws.cloud.qdrant.io:6333", 
    api_key=Qdrant_api_key,
)

print(client.get_collections())

collections=[]


In [3]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

In [4]:
encoder = SentenceTransformer("all-MiniLM-L6-v2")

In [6]:
client.create_collection(
    collection_name="scholarship-vdb",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(),  # Vector size is defined by used model
        distance=models.Distance.COSINE,
    ),
)

True

In [ ]:
client.upload_points(
    collection_name="my_books",
    points=[
        models.PointStruct(
            id=idx, vector=encoder.encode(doc["description"]).tolist(), payload=doc
        )
        for idx, doc in enumerate(documents)
    ],
)

In [ ]:
hits = client.query_points(
    collection_name="my_books",
    query=encoder.encode("alien invasion").tolist(),
    limit=3,
).points

for hit in hits:
    print(hit.payload, "score:", hit.score)